In [1]:
# importing the Keras libraries and packages
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import image_dataset_from_directory


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, Model

# Constants
NUM_CLASSES = 5
IMG_SIZE = 64

# Define the input layer
inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# Add a resizing layer to resize the images to a consistent shape
x = layers.Resizing(IMG_SIZE, IMG_SIZE)(inputs)
# Add a rescaling layer to rescale the pixel values to the [0, 1] range
x = layers.Rescaling(1./255)(x)
# Add some data augmentation layers to apply random transformations during training
x = layers.RandomFlip("horizontal")(x)
# Uncomment the following line if RandomShear is needed
# x = layers.RandomShear(0.2)(x)
x = layers.RandomZoom(0.2)(x)

# Add the convolutional and pooling layers
x = layers.Conv2D(32, (3, 3), activation="relu")(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Dropout(0.2)(x)
x = layers.Conv2D(32, (3, 3), activation="relu")(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Dropout(0.2)(x)

# Flatten the output of the convolutional layers
x = layers.Flatten()(x)
# x = layers.GlobalAveragePooling2D()(x)

# Add the dense layers
x = layers.Dense(128, activation="relu")(x)
# SOFT_MAX ALS ACTIVATION
outputs = layers.Dense(NUM_CLASSES, activation="sigmoid")(x)

# Create the model
model_GAP = Model(inputs=inputs, outputs=outputs, name="base_model_project")

# Compile the model
model_GAP.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model_GAP.summary()

Model: "base_model_project"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resizing (Resizing)             │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip (RandomFlip)        │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_zoom (RandomZoom)        │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       802,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 813,733 (3.10 MB)

 Trainable params: 813,733 (3.10 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
# Set the parameters for your data
batch_size = 32
image_size = (64, 64)
validation_split = 0.3

# Create the training dataset from the 'train' directory
train_ds = image_dataset_from_directory(
    directory='./train/train',
    labels='inferred',
    label_mode='categorical',
    batch_size=batch_size,
    image_size=image_size,
    validation_split=validation_split,
    subset='training',
    seed=123
)

# Create the validation dataset from the 'train' directory
validation_ds = image_dataset_from_directory(
    directory='./train/train',
    labels='inferred',
    label_mode='categorical',
    batch_size=batch_size,
    image_size=image_size,
    validation_split=validation_split,
    subset='validation',
    seed=123
)

# Create the testing dataset from the 'test' directory
test_ds = image_dataset_from_directory(
    directory='./test/',
    labels='inferred',
    label_mode='categorical',
    batch_size=batch_size,
    image_size=image_size
)

# Ensure the dataset can generate enough batches (so keep on generating when needed)
train_ds = train_ds.repeat()
validation_ds = validation_ds.repeat()
test_ds = test_ds.repeat()

Found 2026 files belonging to 5 classes.
Using 1419 files for training.
Found 2026 files belonging to 5 classes.
Using 607 files for validation.
Found 473 files belonging to 1 classes.


In [4]:
model_GAP.fit(train_ds,
    validation_data = validation_ds,
    steps_per_epoch = 100,
    validation_steps = 30, # we have a .20 validation split, so we'll use 80% less batches for validation
    epochs = 8
    )

Epoch 1/8
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.2595 - loss: 1.6262 - val_accuracy: 0.4129 - val_loss: 1.3854
Epoch 2/8
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.4179 - loss: 1.4029 - val_accuracy: 0.3685 - val_loss: 1.6279
Epoch 3/8
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.5297 - loss: 1.2281 - val_accuracy: 0.4578 - val_loss: 1.3930
Epoch 4/8
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.5862 - loss: 1.0862 - val_accuracy: 0.5595 - val_loss: 1.1949
Epoch 5/8
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.6696 - loss: 0.8897 - val_accuracy: 0.5871 - val_loss: 1.1859
Epoch 6/8
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.7157 - loss: 0.7727 - val_accuracy: 0.5365 - val_loss: 1.1881
Epoch 7/8
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.7822 - loss: 0.6310 - val_accuracy: 0.6482 - val_loss: 0.9757
Epoch 8/8
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.7845 - loss: 0.6142 - val_accuracy: 0.